# Calculate Q-Rookley

- Input data "20172022_processed_1_3_4.csv", from 2017/7/1 to 2022/12/31, same as used in Figlewski

- Use 500 rolling window for P density (when calculating EPK)

- Use logreturn instead of log+1

- Use 
```
r.source('Q_Rookley_main_3.R')
```
and
```
'EPK_library_1.r'
```
to calculate Q_Rookley, along with 
```
get_btc_prices_2015_2022
```
for physical density.

- Interest rate: 0

- Moneyness range is [min moneyness, max moneyness]

- Bandwidth 0.3

- tau=29

In [1]:
import platform
plat = platform.system()
import locale
if plat == 'Darwin':
    print('Using Macos Locale')
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
else:
    locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import datetime
import pandas as pd
import sys
import matplotlib.pyplot as plt
import csv
import gc
import math
import numpy as np
import os
import pdb
import pickle
import statsmodels.api as sm
import time


import rpy2.robjects.packages as rpackages
from rpy2 import robjects

from scipy import stats
from scipy.integrate import simps
from scipy import interpolate

from src.helpers import save_dict, load_dict
from src.brc import BRC
from src.strategies import IronCondor
from src.quandl_data import get_btc_prices_2015_2022

from numpy.lib.function_base import insert

from statsmodels.nonparametric.kernel_regression import KernelReg as nadarajawatson

from sklearn.neighbors import KernelDensity

from pathlib import Path

def filter_sub(_df,  mindate, maxdate, tau):
    # Subset

    # Only calls, tau in [0, 0.25] and fix one day (bc looking at intra day here)
    # (_df['is_call'] == 1) & --> Dont need to filter for calls because that happens in the R script
    # Also need to consider put-call parity there
    sub = _df[(_df['date'] >= mindate) & (_df['date'] <= maxdate) & (_df['moneyness'] >= 0.5) & (_df['moneyness'] < 2) &(_df['mark_iv'] > 0)]# &

    if tau > 0:
        sub = sub[sub['tau'] == tau]

    nrows = sub.shape[0]
    if nrows == 0:
        raise(ValueError('Sub is empty'))
   
    sub['moneyness'] = round(sub['moneyness'], 3)
    sub['underlying_price'] = round(sub['underlying_price'], 2)

    sub = sub.drop_duplicates()
    
    print(sub.describe())

    if nrows > 50000:
        print('large df, subsetting')
        sub = sub.sample(10000)
        print(sub.describe())

    return sub

def prepare_confidence_band_data(df):

            # Old
                # Save for Confidence Band 
                    # Column 1 - Day
                    # Column 2 - Month
                    # Column 3 - Year
                    # Column 4 - Call Indicator
                    # Column 5 - Time Maturity in Days
                    # Column 6 - Strike
                    # Column 7 - IV
                    # Column 8 - Spot
                    # Column 9 - Risk Free Rate
                    # Column 10 - 
                    # Column 11 - Forward
                # 3	1	1997	0	49	3025	165.8	2861.706	0.031465	0.150026951	2863.26
                # Day Month Year Put Days Strike IV     Spot         Risk Free    Tau in Days (255)       Forward

    """
    # Mapping Table for Input

    # Column ... X
    # 1 - date
    # 2 - IV
    # 3 - 1 for Call, 0 for Put
    # 4 - Tau 
    # 5 - Strike
    # 6 - assigned meanwhile as put2call[,6]+ put2call[,7] -  put2call[,5]*exp(- mean(put2call[,8])* tau);
    # 7 - Spot
    # 8 - Probably Interest Rate
    # 9 - assigned meanwhile Moneyness: Spot/Strike
    """
    b = df[['date', 'mark_iv', 'putcall', 'tau', 'K', 'underlying_price', 'interest_rate', 'moneyness', 'instrument_price']]
    b['placeholder'] = 0
    b = b.reset_index(drop=True)
    # b['interest_rate'] = 0.00
    b['forward'] = 1

    # Reorder
    df_c = b[['date', 'mark_iv', 'putcall', 'tau', 'K', 'instrument_price', 'underlying_price', 'interest_rate', 'moneyness']]

    unique_maturities = df.days_to_maturity.unique()[0]
    fname = str('confidence_band_input_') + str(unique_maturities) + str('.csv')
    df_c.to_csv(fname, index = False)
    return fname

def bootstrap(conf_fname, phys_fname, rdate, tau, simmethod, r, out_dir, bw, lower, upper):
                            
    try:
        # MAKE SURE TO USE DIFFERENT 2ND INPUT FOR SP500!!
        moneyness, spd, epk, epk_lo, epk_up, logret, spd_logret, epk_logret, epk_logret_lo, epk_logret_up, epk_BS, tau, volas, spd_lo, spd_up, cdf_m, cdf_ret= r.bootstrap_epk(robjects.StrVector([conf_fname]), 
                                      robjects.StrVector([phys_fname]),
                                      robjects.StrVector(rdate),
                                      robjects.StrVector([out_dir]),
                                      robjects.FloatVector([bw]),
                                      robjects.FloatVector([lower]),
                                      robjects.FloatVector([upper])
                                     )
        # Can recover PD here as SPD/EPK
        spd_df = pd.DataFrame({'m': moneyness,
                               'spdy': spd,
                               'spdy_up': spd_up,
                               'spdy_dn': spd_lo,
                               'epk': epk,
                               'ret': logret,
                               'spd_ret': spd_logret,
                               'cb_ret_up': epk_logret_up,
                               'cb_ret_dn': epk_logret_lo,
                               'epk_ret': epk_logret,
                               'volatility': volas,
                               'epk_up': epk_up,
                               'epk_dn': epk_lo,
                               'cdf_m': cdf_m,
                               'cdf_ret': cdf_ret})

        return spd_df
    
    except Exception as e:
        print('Sim failed, proceeding with the next one... - ', simmethod)
        print('exception: ', e)
        
# Initiate R Objects
base = rpackages.importr("base") 
r = robjects.r
r.source('src/confidence_bands.R')
r.source('src/Q_Rookley_main_3.R')

# BTC prices
tdat = get_btc_prices_2015_2022()
tdat[::-1].to_csv('Data/BTC_USD_Quandl_2015_2022.csv', index = False)

bw_range = np.array([0.3])
r_bandwidth = bw_range[0]

simmethod = ['oldschool']

df = pd.read_csv('Data/20172022_processed_1_3_4.csv')
interest_rate_data = pd.read_csv('Data/IR_daily.csv')
interest_rate_data = interest_rate_data.rename(columns={'index': 'date','DTB3': 'interest_rate'})

df1=df[df['tau']==29]
dates=df1['date'].unique()
#df1=df[df['date']>=dates[9]]
#dates = df1['date'].unique()

dates1 = dates

error_i = []
for i0 in range(dates1.size):
    df2 = df1[df1['date']==dates1[i0]]
    
    curr_day = datetime.datetime.strptime(dates1[i0],'%Y-%m-%d')
    curr_day_starttime = curr_day.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
    curr_day_endtime = curr_day.replace(hour = 23, minute = 59, second = 59, microsecond = 0)
    print(curr_day)

    df3 = pd.merge(df2, interest_rate_data, on='date', how='left')
    

    df3['interest_rate'] = 0
    df3['days_to_maturity'] = df3['tau']
    df3['tau'] = df3['tau']/365
    df3['underlying_price'] = df3['BTC_price']
    df3['instrument_price'] = df3['option_price']
    df3['putcall'][df3['putcall']=='C']=1
    df3['putcall'][df3['putcall']=='P']=0
    
    df_1 = df3[['date','BTC_price', 'K', 'interest_rate', 'tau', 'putcall', 'IV',  'moneyness',
                'days_to_maturity',  'underlying_price', 'instrument_price']]
    df_1['mark_iv']=df_1['IV']/100
    
    df_1.to_csv('Results/option_' + datetime.datetime.strftime(curr_day_starttime, '%Y%m%d') + '.csv')
    
    conf_fname = prepare_confidence_band_data(df_1)
    
    rdate = base.as_Date(curr_day.strftime('%Y-%m-%d'))
    rdate_f = base.format(rdate, '%Y-%m-%d')
    
    tau = df['tau'].unique()[0]
    
    spd_btc = bootstrap(conf_fname, 'Data/BTC_USD_Quandl_2015_2022.csv', rdate_f, tau, simmethod, r, 
                                     'Results/Rookley_Q_CB_1722_1_2_5_1_ttm29', r_bandwidth, df_1['moneyness'].min(), df_1['moneyness'].max())
    
    try :
        
        spd_btc.to_csv('Results/Rookley_Q_CB_1722_1_2_5_1_ttm29/btc_pk_'  + str(curr_day_starttime.__format__('%Y-%m-%d')) +'_bw_' + str(r_bandwidth) +  '.csv')
        
        print(spd_btc)
        
    except Exception as e:
        error_i.append(i0)

Using Macos Locale


/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
R[write to console]: 
Attaching package: ‘matlab’


R[write to console]: The following object is masked from ‘package:stats’:

    reshape


R[write to console]: The following objects are masked from ‘package:utils’:

    find, fix


R[write to console]: The following object is masked from ‘package:base’:

    sum


R[write to console]: KernSmooth 2.23 loaded
Copyright M. P. Wand 1997-2009

2017-08-31 00:00:00


/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_25088/2130152264.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_25088/2130152264.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_25088/2130152264.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] "bandwidth: 0.3"
[1] "test no arbitrage condition"
[1] 44.694713  4.722982 66.258718
                [,1]      [,2]         [,3]        [,4]
sigmas -0.7541584942 1.7496801 -5.763566316  18.9262296
       -0.7465839009 1.7281143 -5.683588455  18.7391416
       -0.7390662510 1.7068886 -5.604753705  18.5553256
       -0.7316046948 1.6859967 -5.527037317  18.3747262
       -0.7241984012 1.6654324 -5.450414941  18.1972880
       -0.7168465579 1.6451899 -5.374862638  18.0229558
       -0.7095483699 1.6252632 -5.300356876  17.8516749
       -0.7023030597 1.6056468 -5.226874538  17.6833905
       -0.6951098667 1.5863350 -5.154392919  17.5180481
       -0.6879680464 1.5673224 -5.082889734  17.3555937
       -0.6808768703 1.5486039 -5.012343112  17.1959735
       -0.6738356250 1.5301742 -4.942731601  17.0391342
       -0.6668436125 1.5120284 -4.874034165  16.8850227
       -0.6599001489 1.4941617 -4.806230183  16.7335865
       -0.6530045647 1.4765692 -4.739299451  16.5847735
       -0.64615

...

[1] "bandwidth: 0.3"
[1] "test no arbitrage condition"
[1] 44.694713  4.722982 66.258718
               [,1]      [,2]         [,3]         [,4]
sigmas -0.642390826 1.2614742 -2.026380771  4.004233426
       -0.628377809 1.2466198 -1.997479080  3.985690810
       -0.614558446 1.2319738 -1.968681855  3.966987749
       -0.600927458 1.2175356 -1.939991417  3.948127194
       -0.587479778 1.2033047 -1.911410035  3.929112048
       -0.574210540 1.1892806 -1.882939925  3.909945173
       -0.561115072 1.1754627 -1.854583257  3.890629379
       -0.548188881 1.1618504 -1.826342147  3.871167430
       -0.535427647 1.1484431 -1.798218663  3.851562039
       -0.522827211 1.1352401 -1.770214827  3.831815870
       -0.510383573 1.1222409 -1.742332610  3.811931534
       -0.498092878 1.1094446 -1.714573938  3.791911590
       -0.485951412 1.0968505 -1.686940689  3.771758545
       -0.473955594 1.0844580 -1.659434698  3.751474850
       -0.462101972 1.0722662 -1.632057755  3.731062905
       -0.45038

        0.462192706 0.7561904  0.440987896  0.414032149
        0.466857690 0.7594803  0.443774128  0.384349727
        0.471501013 0.7627893  0.446334752  0.354634392
        0.476122876 0.7661158  0.448669694  0.324887850
        0.480723475 0.7694579  0.450778905  0.295111872
        0.485303005 0.7728141  0.452662370  0.265308292
        0.489861659 0.7761825  0.454320107  0.235479009
        0.494399625 0.7795617  0.455752168  0.205625988
        0.498917092 0.7829497  0.456958638  0.175751260
        0.503414243 0.7863451  0.457939639  0.145856924
        0.507891260 0.7897460  0.458695333  0.115945146
        0.512348322 0.7931508  0.459225916  0.086018158
        0.516785608 0.7965579  0.459531628  0.056078264
        0.521203290 0.7999656  0.459612747  0.026127832
        0.525601543 0.8033721  0.459469595 -0.003830699
        0.529980536 0.8067760  0.459102536 -0.033794824
        0.534340436 0.8101754  0.458511979 -0.063761968
        0.538681411 0.8135689  0.457698378 -0.09

/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_25088/2130152264.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_25088/2130152264.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_25088/2130152264.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]         [,3]        [,4]
sigmas -0.713153778 1.2016652 -2.479896927  5.73577954
       -0.698524554 1.1841279 -2.440450364  5.70519404
       -0.684106263 1.1668614 -2.401155693  5.67429830
       -0.669892908 1.1498652 -2.362016892  5.64309647
       -0.655878745 1.1331386 -2.323037882  5.61159269
       -0.642058267 1.1166809 -2.284222529  5.57979107
       -0.628426193 1.1004915 -2.245574646  5.54769572
       -0.614977457 1.0845696 -2.207097990  5.51531072
       -0.601707192 1.0689144 -2.168796265  5.48264014
       -0.588610722 1.0535251 -2.130673120  5.44968803
       -0.575683556 1.0384009 -2.092732149  5.41645841
       -0.562921370 1.0235409 -2.054976892  5.38295530
       -0.550320008 1.0089441 -2.017410836  5.34918266
       -0.537875466 0.9946095 -1.980037414  5.31514447
       -0.525583889 0.9805362 -1.942860007  5.28084466
       -0.513441562 0.9667231 -1.905881940  5.24628715
       -0.501444904 0.9531691 -

        0.439410828 0.7187751  0.505809187 -0.26547487
        0.444054277 0.7223861  0.504140157 -0.30409180
        0.448676264 0.7259861  0.502225886 -0.34248012
        0.453276986 0.7295736  0.500069402 -0.38063618
        0.457856639 0.7331471  0.497673759 -0.41855650
        0.462415414 0.7367052  0.495042037 -0.45623771
        0.466953501 0.7402465  0.492177340 -0.49367661
        0.471471087 0.7437698  0.489082787 -0.53087012
        0.475968355 0.7472736  0.485761511 -0.56781534
        0.480445489 0.7507567  0.482216657 -0.60450952
        0.484902668 0.7542178  0.478451372 -0.64095010
        0.489340068 0.7576557  0.474468809 -0.67713468
        0.493757865 0.7610693  0.470272114 -0.71306106
        0.498156230 0.7644573  0.465864428 -0.74872720
        0.502535335 0.7678187  0.461248879 -0.78413130
        0.506895347 0.7711522  0.456428579 -0.81927173
        0.511236431 0.7744570  0.451406619 -0.85414706
        0.515558752 0.7777318  0.446186066 -0.88875610
        0.

In [2]:
print(error_i)

[2, 3, 5, 6, 8, 9, 10, 12]
